# Creating a JSO compatible solver
### João Okimoto

![](../../jso.png)

In [ ]:
using Pkg
pkg"activate ."
if false
 pkg"add NLPModels"
 pkg"add SolverTools"
 pkg"add SolverBenchmark"
 pkg"add Plots"
 pkg"add LinearAlgebra"
 pkg"add JSOSolvers"
end
pkg"instantiate"
pkg"status"

This tutorial covers the basics on how to create a solver compatible with the
JuliaSmoothOptimizers format and do basic benchmarking.
Let's start off with a simple implementation of Newton's method:

In [ ]:
using NLPModels, LinearAlgebra, SolverTools

function newton(nlp :: AbstractNLPModel;
                x :: AbstractVector = copy(nlp.meta.x0),
                max_tol :: Real = √eps(eltype(x)),
                max_time :: Float64 = 30.0,
                max_iter :: Int = 100)

  T = eltype(x)
  k = 0
  el_time = 0.0
  start_time = time()
  tired = el_time > max_time || k ≥ max_iter
  optimal = norm(grad(nlp, x)) < max_tol

  @info log_header([:iter, :f, :nrmgrad], [Int, T, T], hdr_override=Dict(:f => "f(x)", :nrmgrad => "‖∇f(x)‖"))

  while !(optimal || tired)

    fx = obj(nlp, x)
    ∇fx = grad(nlp, x)
    nrmgrad = norm(∇fx)
    ∇²fx = Symmetric(hess(nlp, x), :L)

    @info log_row(Any[k, fx, nrmgrad])

    d = isposdef(∇²fx) ? ∇²fx\ -∇fx : -∇fx
    t = one(T)
    α = 0.5

    while obj(nlp, x + t*d) > fx + α * ∇fx' * t * d
      t *= 0.5
    end

    x += t*d

    k += 1
    el_time = time() - start_time
    tired = el_time > max_time || k ≥ max_iter
    optimal = nrmgrad < max_tol
  end

  if optimal
    status =:first_order
  elseif tired
    if k ≥ max_iter
      status =:max_iter
    else
      status =:max_time
    end
  else
    status =:unknown
  end

  return GenericExecutionStats(status, nlp, solution=x, objective=obj(nlp, x),
                               iter = k, elapsed_time = el_time)

end

As you can see in the example above, our ```newton``` solver has a few important
features:
- It receives the **JSO Standard input**, an **AbtractNLPModel**, part of the [NLPModels](https://github.com/JuliaSmoothOptimizers/NLPModels.jl) package.
- Derivatives are computed according to the **AbstractNLPModel** implementation with the ``hess`` and ``grad`` functions.
- It returns the **JSO Standard output**, a **GenericExecutionStats**, part of the [SolverTools](https://github.com/JuliaSmoothOptimizers/SolverTools.jl) package.

Both input and output are important when interacting with other JSO packages,
since they serve as a way of solving a optimization problem while also keeping
track of relevant information. An **AbstractNLPModel** is the main structure of
the problem, as it contains details regarding its description (i.e function to
be minimized, starting point, number of variables, etc.), while a
**GenericExecutionStats** is responsible for carrying information over how the
problem was solved (i.e number of iterations, time elapsed, number of function
evaluations, etc.).

Another important thing to highlight, is that we'll be using the **ADNLPModel**
struct to run our solver. By default, the package responsible for calculating
derivatives is [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl) since
that's the built in method for differentation in this **AbstractNLPModel**
implementation.

When creating a new JSO compatible solver, it's also important to manipulate
the output data to match that of **GenericExecutionStats**. The ``status``
attribute tells us about how our solver did given our input problem. It is
possible that the number of iterations has exceed the max limit, or the time it
took for it to solve was greater than allowed. Either way, the ``status`` is an
important attribute when benchmarking and testing, as it allows the user to keep
track of how a solver behaves over a specific problem.

If we want to run our solver, we simply need to create a optimization problem

In [ ]:
f(x) = (x[1]^2 + x[2]^2)^2
  problem = ADNLPModel(f, [1.0,2.0])
  output = newton(problem)

whose information regarding its solution is stored in ``output``.

In [ ]:
print(output)

With a solver in hands, we can start to do more interesting things, such as
benchmarking and comparing our ```newton``` method to other solvers. Since JSO
contains a lot of useful packages, we'll be using a collection of those to make
our comparison easier. The most important one by far is [SolverBenchmarks](https://github.com/JuliaSmoothOptimizers/SolverBenchmark.jl), a
benchmarking package designed for manipulating DataFrames that contain statistics
regarding the execution of a specific sover given a set of problems.

Let us compare our ```newton``` method to a solver contained in the
[JSOSolvers](https://github.com/JuliaSmoothOptimizers/JSOSolvers.jl) package, namely the ```lbfgs``` one.
Firstly, we'll need to gather our data regarding each solver.

In [ ]:
using SolverBenchmark, JSOSolvers
solvers = Dict(:newton=>newton, :lbfgs=>lbfgs)
f1(x) = x[1]^2 + x[2]^2
f2(x) = (1 - x[1])^2 + 100(x[2] - x[1]^2)^2
f3(x) = (x[1]^2 + x[2] - 11) + (x[1] + x[2]^2 - 7)^2
f4(x) = 0.26(x[1]^2 + x[2]^2) - 0.48*x[1]*x[2]
test_functions = [f1, f2, f3, f4]
problems = (ADNLPModel(i, 2*ones(2),name="Problem $i") for i in test_functions)
stats = bmark_solvers(solvers, problems)

The ``bmark_solvers`` function is also part of the **SolverTools** package, and
runs our solver on a set of problems, in our case ``f1``, ``f2``, ``f3`` and ``f4``.
Lastly, it returns a DataFrame of statistics that we can manipulate using
**SolverBenchmarks**.

Next, let's create a latex table with the data we just gathered

In [ ]:
open("newton.tex","w") do io
	latex_table(io,stats[:newton],cols = [:name,:status,:objective,:elapsed_time,:iter])
end
open("lbfgs.tex","w") do io
	latex_table(io,stats[:lbfgs],cols = [:name,:status,:objective,:elapsed_time,:iter])
end

That will give us a nicely formatted table that we can just plug into our
latex code. It's also possible to create the table in the markdown format.

In [ ]:
open("newton.md","w") do io
	markdown_table(io,stats[:newton],cols = [:name,:status,:objective,:elapsed_time,:iter])
end
open("lbfgs.md","w") do io
	markdown_table(io,stats[:lbfgs],cols = [:name,:status,:objective,:elapsed_time,:iter])
end

Either way, our tables should look something like this:
### LBFGS

```
|       name |      status | objective | elapsed_time | iter |
|------------|-------------|-----------|--------------|------|
| Problem f1 | first_order |   0.0e+00 |      1.2e+00 |    2 |
| Problem f2 | first_order |   1.3e-16 |      7.0e-04 |   25 |
| Problem f3 | first_order |  -8.4e+00 |      3.7e-04 |    8 |
| Problem f4 | first_order |   1.2e-28 |      9.8e-05 |    2 |
```
### Newton

```
|       name |      status | objective | elapsed_time | iter |
|------------|-------------|-----------|--------------|------|
| Problem f1 | first_order |   0.0e+00 |      6.1e-01 |    2 |
| Problem f2 | first_order |   0.0e+00 |      1.5e-01 |   17 |
| Problem f3 | first_order |  -8.4e+00 |      1.5e-01 |   16 |
| Problem f4 | first_order |   5.6e-62 |      1.4e-01 |    2 |
```
In the latex format

![](lbfgs.svg)

![](newton.svg)

To create a performance profile, we use the **Plots** package, along with the desired framework.

In [ ]:
using Plots
plotly()
performance_profile(stats, df->df.elapsed_time)